*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer 
print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [89]:
df_train = pd.read_csv(train_file_path, sep='\t', header=0,
    names=['h_or_s', 'text'])
df_test = pd.read_csv(test_file_path, sep='\t', header=0,
    names=['h_or_s', 'text'])
df_train.head()
df_test.head()

,h_or_s,text
0,ham,"not much, just some textin'. how bout you?"
1,ham,i probably won't eat at all today. i think i'm...
2,ham,don‘t give a flying monkeys wot they think and...
3,ham,who are you seeing?
4,ham,your opinion about me? 1. over 2. jada 3. kusr...


In [81]:
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [90]:
maxLen = 50
vocabSize = 40000
df_train.isnull().sum()
df_train['h_or_s'] = [0 if i=='ham' else 1 for i in df_train['h_or_s']]

In [91]:
print(df_train.groupby('h_or_s').count())
print(df_test.groupby('h_or_s').count())

        text
h_or_s      
0       3618
1        560
        text
h_or_s      
ham     1204
spam     187


In [92]:
cleanText = []
for sentence in df_train['text']:
  k = ''
  for i in sentence:
    if i not in string.punctuation:
      k += i
  cleanText.append(k)
df_train['text'] = cleanText

In [93]:
# print(df_train.groupby(['h_or_s','text']).size().reset_index(name='Count'))
print(len(df_train))
print(len(df_train[['h_or_s','text']].drop_duplicates()))

4178
3916


In [94]:
df_train = df_train[['h_or_s','text']].drop_duplicates()
print(df_train.groupby('h_or_s').count())
print(df_test.groupby('h_or_s').count())

        text
h_or_s      
0       3418
1        498
        text
h_or_s      
ham     1204
spam     187


In [95]:
df_train.head()
df_train['text'] = [i.split() for i in df_train['text']]

In [96]:
df_train.head()
cleanText = []
for sentences in df_train['text']:
  k = ''
  for words in sentences:
    if words.isalpha():
      # if words.lower() in stopwords.words('english'):
      k += words + " "
  cleanText.append(k)
df_train['ctext'] = cleanText
df_train.head()

,h_or_s,text,ctext
0,0,"[you, can, never, do, nothing]",you can never do nothing
1,0,"[now, u, sound, like, manky, scouse, boy, stev...",now u sound like manky scouse boy stevelike i ...
2,0,"[mum, say, we, wan, to, go, then, go, then, sh...",mum say we wan to go then go then she can shun...
3,0,"[never, y, lei, i, v, lazy, got, wat, dat, day...",never y lei i v lazy got wat dat day ü send me...
4,0,"[in, xam, hall, boy, asked, girl, tell, me, th...",in xam hall boy asked girl tell me the startin...


In [97]:
df_train['length1'] = [len(i) for i in df_train['text']]
df_train.describe()
# for i in range(len(df_train)):
#   print(df_train.iloc[i])
#   break
# df_train.query("h_or_s == 0").sample(n=2)
# df_train1 = df_train[ledf_train['ctext']]

,h_or_s,length1
count,3916.000000,3916.000000
mean,0.127171,15.277324
std,0.333207,10.975147
min,0.000000,0.000000
25%,0.000000,7.000000
50%,0.000000,12.000000
75%,0.000000,22.000000
max,1.000000,171.000000


In [98]:
xyz = df_train.query("h_or_s == 0").sample(n=550)
abc = df_train.query("h_or_s == 1").sample(n=498)
xyz = xyz.append(abc)
df_train_f = xyz.sample(frac = 1)
df_train_f.head()

,h_or_s,text,ctext,length1
734,1,"[free, video, camera, phones, with, half, pric...",free video camera phones with half price line ...,24
91,0,"[i, dont, know, jack, shit, about, anything, o...",i dont know jack shit about anything or id say...,32
985,1,"[dear, matthew, please, call, 09063440451, fro...",dear matthew please call from a landline your ...,23
2534,1,"[monthly, password, for, wap, mobsicom, is, 39...",monthly password for wap mobsicom is use your ...,13
3948,0,"[i, gotta, collect, da, car, at, 6, lei]",i gotta collect da car at lei,8


In [99]:
print(df_train_f.groupby('h_or_s').count())

        text  ctext  length1
h_or_s                      
0        550    550      550
1        498    498      498


In [100]:
tokenizer = Tokenizer(num_words=vocabSize)#, oov_token='<UNK>')
tokenizer.fit_on_texts(df_train_f['ctext'])
word_index = tokenizer.word_index
# print(df_train.head())
trainSequence = tokenizer.texts_to_sequences(df_train_f['ctext'])
print(trainSequence)
trainPadded = pad_sequences(trainSequence, maxlen=maxLen, padding='post', truncating='post')
print(trainPadded)
# testSequence = tokenizer.texts_to_sequences(df_test['text'])
# testPadded = pad_sequences(testSequence, maxlen=maxLen, padding='post', truncating='post')

[[14, 104, 139, 435, 21, 227, 192, 113, 306, 9, 1332, 11, 934, 721, 105, 436, 6, 12], [5, 60, 72, 1333, 381, 83, 437, 12, 503, 1334, 307, 1335, 51, 49, 2, 67, 2, 50, 1336, 38, 5, 206, 11, 25, 26, 20, 722, 17, 604, 1, 4, 1337], [118, 1338, 61, 6, 23, 3, 126, 7, 382, 438, 114, 12, 69, 246, 156, 228, 136], [1339, 935, 9, 439, 1340, 8, 308, 7, 439, 63, 43, 605], [5, 606, 147, 157, 723, 46, 724], [440, 2, 47, 193, 73, 35, 1341, 725, 1342, 11, 936, 35, 340, 32, 4, 937, 19, 7, 607, 274, 2, 24, 97, 1343, 1344, 2, 1345, 194, 35, 24, 1346, 341, 1, 1347, 195], [1348, 101, 1349, 5, 50, 32, 179], [196, 10, 67, 20, 60, 10, 608, 41, 1350, 11, 15, 3, 504, 1351, 383, 3, 1352, 65, 21, 3, 247, 1353, 79, 83, 35, 40, 1354, 98, 609, 22, 275], [247, 1355, 1356, 84, 309, 19, 1357, 309, 1, 60, 180, 48, 80, 127, 229, 3, 938, 115, 310, 137, 939], [1358, 24, 1359, 206, 2, 1360], [128, 40, 181, 505, 311, 726, 1, 384, 20, 52, 46], [2, 24, 506, 610, 66, 34, 230, 70, 66, 197, 2, 72, 1, 108, 48, 57, 28, 8, 6, 23, 7, 3

In [101]:
print(np.asarray(trainPadded))

[[  14  104  139 ...    0    0    0]
 [   5   60   72 ...    0    0    0]
 [ 118 1338   61 ...    0    0    0]
 ...
 [  30  177  305 ...    0    0    0]
 [  78  178   41 ...    0    0    0]
 [3235   64 3236 ...    0    0    0]]


In [102]:
model = ''
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocabSize, 64, input_length=maxLen),
                             tf.keras.layers.LSTM(64),
                             tf.keras.layers.Dense(1, activation="sigmoid")
])
model.summary()
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=['accuracy'])
df_train_f.head()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 50, 64)            2560000   
                                                                 
 lstm_5 (LSTM)               (None, 64)                33024     
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2,593,089
Trainable params: 2,593,089
Non-trainable params: 0
_________________________________________________________________


,h_or_s,text,ctext,length1
734,1,"[free, video, camera, phones, with, half, pric...",free video camera phones with half price line ...,24
91,0,"[i, dont, know, jack, shit, about, anything, o...",i dont know jack shit about anything or id say...,32
985,1,"[dear, matthew, please, call, 09063440451, fro...",dear matthew please call from a landline your ...,23
2534,1,"[monthly, password, for, wap, mobsicom, is, 39...",monthly password for wap mobsicom is use your ...,13
3948,0,"[i, gotta, collect, da, car, at, 6, lei]",i gotta collect da car at lei,8


In [103]:
history = ''
history = model.fit(np.asarray(trainPadded), df_train_f['h_or_s'], epochs=5, batch_size=32, verbose=True)

Epoch 1/5
33/33 [==============================] - 4s 60ms/step - loss: 0.6928 - accuracy: 0.5248
Epoch 2/5
33/33 [==============================] - 2s 60ms/step - loss: 0.5026 - accuracy: 0.7529
Epoch 3/5
33/33 [==============================] - 2s 60ms/step - loss: 0.1582 - accuracy: 0.9609
Epoch 4/5
33/33 [==============================] - 2s 60ms/step - loss: 0.0769 - accuracy: 0.9857
Epoch 5/5
33/33 [==============================] - 2s 61ms/step - loss: 0.1083 - accuracy: 0.9781


In [104]:
p_t = tokenizer.texts_to_sequences(pd.Series(["sale today! to stop texts call 98912460324"]))
p_t_padded = pad_sequences(p_t, maxlen=maxLen, padding='post', truncating='post')
sheesh = model.predict(p_t_padded)
if round(sheesh[0][0])==1:
  predic = 'spam'
else:
  predic = 'ham'
print(predic,sheesh)

1/1 [==============================] - 0s 480ms/step
spam [[0.9725666]]


In [105]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  p_t = tokenizer.texts_to_sequences(pd.Series([pred_text]))
  p_t_padded = pad_sequences(p_t, maxlen=maxLen, padding='post', truncating='post')
  sheesh = model.predict(p_t_padded)
  if round(sheesh[0][0])==1:
    predic = 'spam'
  else:
    predic = 'ham'
  prediction = [sheesh[0][0], predic]
  print(prediction)
  return (prediction)
pred_text = "how are you doing today?"
prediction = predict_message(pred_text)[0]
print(prediction)

1/1 [==============================] - 0s 27ms/step
[0.011065427, 'ham']
0.011065427


In [106]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 28ms/step
[0.011065427, 'ham']
1/1 [==============================] - 0s 24ms/step
[0.9725666, 'spam']
1/1 [==============================] - 0s 24ms/step
[0.01071676, 'ham']
1/1 [==============================] - 0s 30ms/step
[0.972568, 'spam']
1/1 [==============================] - 0s 29ms/step
[0.97256976, 'spam']
1/1 [==============================] - 0s 27ms/step
[0.010668503, 'ham']
1/1 [==============================] - 0s 26ms/step
[0.0107128685, 'ham']
You passed the challenge. Great job!
